In [4]:
!pip install datasets
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [9]:
import torch
import numpy as np
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load AG News dataset
dataset = load_dataset("ag_news")

# Initialize model and tokenizer
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4,
    torch_dtype=torch.float32  # Changed to float32
).to(device)

# Configure LoRA
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"]
)

# Create PEFT model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Prepare smaller dataset
train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
eval_dataset = dataset["test"].shuffle(seed=42).select(range(200))

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

# Convert datasets to PyTorch format
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_eval.set_format("torch", columns=["input_ids", "attention_mask", "label"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./peft_results",
    learning_rate=1e-3,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./peft_logs',
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    fp16=False,  # Disabled mixed precision training
    remove_unused_columns=False
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
)

# Evaluate before training
print("Metrics before fine-tuning:")
initial_metrics = trainer.evaluate()
print(initial_metrics)

# Train
print("\nFine-tuning with LoRA...")
trainer.train()

# Final evaluation
print("\nMetrics after fine-tuning:")
final_metrics = trainer.evaluate()
print(final_metrics)

# Print improvements
print("\nImprovement Summary:")
for metric in ['accuracy', 'f1', 'precision', 'recall']:
    initial_value = initial_metrics[f'eval_{metric}']
    final_value = final_metrics[f'eval_{metric}']
    improvement = final_value - initial_value
    print(f"{metric}: {improvement:.4f} improvement (from {initial_value:.4f} to {final_value:.4f})")

# Show model size comparison
original_size = sum(p.numel() * p.element_size() for p in model.parameters()) / (1024 * 1024)  # MB
trainable_size = sum(p.numel() * p.element_size() for p in model.parameters() if p.requires_grad) / (1024 * 1024)  # MB
print(f"\nModel Size Comparison:")
print(f"Full model size: {original_size:.2f} MB")
print(f"Trainable parameters size: {trainable_size:.2f} MB")
print(f"Size reduction: {(1 - trainable_size/original_size)*100:.2f}%")

Using device: cuda


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,036,036 || all params: 125,684,744 || trainable%: 0.8243
Metrics before fine-tuning:


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.3974658250808716, 'eval_model_preparation_time': 0.0337, 'eval_accuracy': 0.225, 'eval_f1': 0.12270571346540338, 'eval_precision': 0.11164750957854405, 'eval_recall': 0.225, 'eval_runtime': 0.9513, 'eval_samples_per_second': 210.244, 'eval_steps_per_second': 13.666}

Fine-tuning with LoRA...


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1,Precision,Recall
1,0.699900,0.704658,0.033700,0.790000,0.792573,0.845637,0.790000
2,0.334800,0.511589,0.033700,0.850000,0.850346,0.866818,0.850000
3,0.217500,0.500315,0.033700,0.860000,0.859845,0.871126,0.860000



Metrics after fine-tuning:


{'eval_loss': 0.5003151297569275, 'eval_model_preparation_time': 0.0337, 'eval_accuracy': 0.86, 'eval_f1': 0.8598447695959499, 'eval_precision': 0.8711260269025911, 'eval_recall': 0.86, 'eval_runtime': 0.6462, 'eval_samples_per_second': 309.491, 'eval_steps_per_second': 20.117, 'epoch': 3.0}

Improvement Summary:
accuracy: 0.6350 improvement (from 0.2250 to 0.8600)
f1: 0.7371 improvement (from 0.1227 to 0.8598)
precision: 0.7595 improvement (from 0.1116 to 0.8711)
recall: 0.6350 improvement (from 0.2250 to 0.8600)

Model Size Comparison:
Full model size: 479.45 MB
Trainable parameters size: 3.95 MB
Size reduction: 99.18%
